# Data Integration

This notebook integrates two datasets from different sources (OLB and VAA) for FPL player data analysis.

OLB: https://github.com/olbauday/FPL-Elo-Insights/tree/main/data/2024-2025

VAA: https://github.com/vaastav/Fantasy-Premier-League/tree/master/data/2024-25

RAN: https://github.com/Randdalf/fplcache/tree/main?tab=readme-ov-file  (honorable mention for understanding how the official fpl api cache works, and for cross referencing)
## Technical Background

- **Challenge**: No open dataset provides a complete time series of players' buying prices
- **Player prices are dynamic**: Driven by transfer in/out (demand and supply)
- **Price fluctuation**: Can rise or fall up to 0.1 Million per day

**Study Period**: 2024-25 season, GW 1 to GW 38

In [1]:
import pandas as pd
import numpy as np
import csv

---

# Part 1: Problem from olbauday (OLB Dataset)

**Datasets used**: `players.csv`, `playerstats.csv`

## Step 1: Map Player IDs to Names

**Problem**: `players.csv` contains id-to-name mapping and `playerstats.csv` uses 'id' to identify players, but using player ID is not compatible with the vaastav dataset.

**Solution**: Map the id to the player's name to make later merging easier.

In [2]:
# Step 1: Read players.csv and create id-to-name mapping
print("Reading players.csv...")
players_df = pd.read_csv('gw_olb/players.csv')

# Create mapping dictionary: player_id -> "first_name second_name"
players_df['full_name'] = players_df['first_name'] + ' ' + players_df['second_name']
id_to_name = dict(zip(players_df['player_id'], players_df['full_name']))
player_ids = set(players_df['player_id'].unique())
print(f"Found {len(player_ids)} distinct player IDs in players.csv")

# Step 2: Read playerstats.csv
print("Reading playerstats.csv...")
playerstats_df = pd.read_csv('gw_olb/playerstats.csv')

# Get distinct IDs from playerstats
playerstats_ids = set(playerstats_df['id'].unique())
print(f"Found {len(playerstats_ids)} distinct IDs in playerstats.csv")

# Step 3: Check if player_ids is a superset of playerstats_ids
is_superset = player_ids >= playerstats_ids

if not is_superset:
    # Find missing IDs
    missing_ids = playerstats_ids - player_ids
    print(f"\nERROR: Superset check FAILED!")
    print(f"Found {len(missing_ids)} IDs in playerstats.csv that are not in players.csv:")
    for missing_id in sorted(missing_ids):
        print(f"  - ID: {missing_id}")
    print("\nStopping script. Please provide instructions on how to proceed.")
else:
    print("\nSuperset check PASSED - All IDs in playerstats.csv exist in players.csv")
    
    # Step 4: Create mapped dataframe
    print("Creating playerstats_mapped.csv...")
    
    # Map the 'id' column to 'player_name'
    playerstats_df['player_name'] = playerstats_df['id'].map(id_to_name)
    
    # Drop the 'id' column and reorder to put player_name first
    columns = ['player_name'] + [col for col in playerstats_df.columns if col not in ['id', 'player_name']]
    mapped_df = playerstats_df[columns]
    
    # Save to new CSV
    mapped_df.to_csv('gw_olb/playerstats_mapped.csv', index=False)
    
    print(f"Successfully created gw_olb/playerstats_mapped.csv")
    print(f"Total rows: {len(mapped_df)}")
    print(f"Columns: {list(mapped_df.columns)}")

Reading players.csv...
Found 804 distinct player IDs in players.csv
Reading playerstats.csv...
Found 804 distinct IDs in playerstats.csv

Superset check PASSED - All IDs in playerstats.csv exist in players.csv
Creating playerstats_mapped.csv...
Successfully created gw_olb/playerstats_mapped.csv
Total rows: 27657
Columns: ['player_name', 'status', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'now_cost', 'now_cost_rank', 'now_cost_rank_type', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'selected_by_percent', 'selected_rank', 'selected_rank_type', 'total_points', 'event_points', 'points_per_game', 'points_per_game_rank', 'points_per_game_rank_type', 'bonus', 'bps', 'form', 'form_rank', 'form_rank_type', 'value_form', 'value_season', 'dreamteam_count', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'ep_next', 'ep_this', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'exp

## Step 2: Filter Playerstats Columns

**Problem**: After mapping id to name, there are too many variables unnecessary for the study.

**Solution**: Keep only the following variables: `player_name`, `status`, `chance_of_playing_next_round`, `chance_of_playing_this_round`, `now_cost`, `event_points`, `ep_next`, `ep_this`, `gw`

**Important Note**: 
- The definition of 'total_points' in OLB's dataset is the **total points accumulated from GW1**
- The definition of 'total_points' in VAA's dataset is the **total points accumulated in that GW**
- After cross-referencing, 'event_points' in OLB's dataset represents the **points the player scores in that particular GW**
- (Points here mean FPL points, not number of goals)

*P.S. The story of this finding is definitely not linear - you can imagine how desperate it is to work with such data in real life.*

In [3]:
# Define the columns to extract
columns_to_extract = [
    'player_name',
    'status',
    'chance_of_playing_next_round',
    'chance_of_playing_this_round',
    'now_cost',
    'event_points',
    'ep_next',
    'ep_this',
    'gw'
]

# Read only the specified columns from the CSV file
df = pd.read_csv('gw_olb/playerstats_mapped.csv', usecols=columns_to_extract)

# Save the filtered data to a new CSV file
df.to_csv('gw_olb/playerstats_mapped_filtered.csv', index=False)

print(f"Successfully extracted {len(columns_to_extract)} columns from playerstats_mapped.csv")
print(f"Output saved to: gw_olb/playerstats_mapped_filtered.csv")
print(f"Number of rows: {len(df)}")

Successfully extracted 9 columns from playerstats_mapped.csv
Output saved to: gw_olb/playerstats_mapped_filtered.csv
Number of rows: 27657


## Step 3: Check for Duplicate (gw, name) Pairs in OLB Dataset

**Context**: In Premier League, it is general to see every club has only one match (fixture) in every gameweek.

**Verification**: Check if `playerstats_mapped_filtered.csv` contains any duplicate (gw=i, name=j) pairs.

**Expected Result**: All pairs (gw, name) should be distinct and unique in this dataset.

**Note**: However, this is not the case for the dataset obtained from vaastav (as we'll see later).

In [4]:
def check_duplicates_olb(file_path, gw_col, name_col, file_description):
    """
    Check for duplicate (gw, name) combinations in a CSV file.
    """
    print(f"\n{'='*80}")
    print(f"Checking: {file_description}")
    print(f"File: {file_path}")
    print(f"Columns: gw='{gw_col}', name='{name_col}'")
    print(f"{'='*80}")
    
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Basic statistics
        total_rows = len(df)
        print(f"\nTotal rows: {total_rows:,}")
        
        # Check for null values
        null_gw = df[gw_col].isnull().sum()
        null_name = df[name_col].isnull().sum()
        
        if null_gw > 0:
            print(f"WARNING: {null_gw} null values found in '{gw_col}' column")
        if null_name > 0:
            print(f"WARNING: {null_name} null values found in '{name_col}' column")
        
        # Group by (gw, name) and count occurrences
        duplicates = df.groupby([gw_col, name_col]).size().reset_index(name='count')
        duplicates = duplicates[duplicates['count'] > 1].sort_values('count', ascending=False)
        
        # Report results
        if len(duplicates) == 0:
            print(f"\n PASS: No duplicate (gw, name) combinations found!")
            unique_combinations = len(df.groupby([gw_col, name_col]))
            print(f"  Total unique (gw, name) combinations: {unique_combinations:,}")
            
            # Show gameweek coverage
            gw_coverage = sorted(df[gw_col].unique())
            print(f"  Gameweeks present: {min(gw_coverage)} to {max(gw_coverage)}")
            print(f"  Number of unique players: {df[name_col].nunique():,}")
            return True
        else:
            print(f"\n FAIL: Found {len(duplicates)} duplicate (gw, name) combinations!")
            print(f"\nDuplicate combinations (showing all):")
            print(f"{'GW':<6} {'Player Name':<40} {'Count':<6}")
            print("-" * 60)
            
            for _, row in duplicates.iterrows():
                print(f"{row[gw_col]:<6} {str(row[name_col])[:40]:<40} {row['count']:<6}")
            
            # Show total affected rows
            total_duplicate_rows = duplicates['count'].sum()
            print(f"\nTotal rows involved in duplicates: {total_duplicate_rows:,}")
            return False
            
    except FileNotFoundError:
        print(f"ERROR: File not found: {file_path}")
        return False
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False

# Check OLB file
olb_ok = check_duplicates_olb(
    file_path='gw_olb/playerstats_mapped_filtered.csv',
    gw_col='gw',
    name_col='player_name',
    file_description='OLB: playerstats_mapped_filtered.csv'
)


Checking: OLB: playerstats_mapped_filtered.csv
File: gw_olb/playerstats_mapped_filtered.csv
Columns: gw='gw', name='player_name'

Total rows: 27,657

 PASS: No duplicate (gw, name) combinations found!
  Total unique (gw, name) combinations: 27,657
  Gameweeks present: 1 to 38
  Number of unique players: 804


---

# Part 2: Problem from vaastav (VAA Dataset)

**Datasets used**: `gw1.csv` to `gw38.csv`

## Step 1: Check GW Column Consistency

**Problem**: Failed to merge gw1 to gw38 initially.

**Investigation**: Found that `gw22.csv` to `gw38.csv` have additional columns.

**Conclusion**: Seems to be a structural change from VAA's data source.

In [5]:
def check_column_consistency():
    # Read reference file (gw1.csv)
    reference_file = 'gw_vaa/gw1.csv'
    with open(reference_file, 'r', encoding='utf-8') as f:
        reference_columns = next(csv.reader(f))
    
    reference_count = len(reference_columns)
    
    # Prepare output
    output_lines = []
    output_lines.append("=== Column Check Report ===")
    output_lines.append(f"Reference: {reference_file}\n")
    
    # Check all files and collect information
    all_files_info = []
    count_mismatches = []
    order_mismatches = []
    
    for i in range(1, 39):  # gw1 to gw38
        filename = f'gw_vaa/gw{i}.csv'
        
        # Read header
        with open(filename, 'r', encoding='utf-8') as f:
            columns = next(csv.reader(f))
        
        column_count = len(columns)
        
        # Store file info
        all_files_info.append({
            'name': f'gw{i}',
            'count': column_count,
            'columns': columns
        })
        
        # Output file info
        columns_str = str(columns)
        output_lines.append(f"{all_files_info[-1]['name']} : number of column = {column_count}, column: {columns_str}")
        
        # Check for differences (skip gw1 as it's the reference)
        if i > 1:
            # Check column count
            if column_count != reference_count:
                missing = set(reference_columns) - set(columns)
                extra = set(columns) - set(reference_columns)
                count_mismatches.append({
                    'file': f'gw{i}',
                    'count': column_count,
                    'missing': list(missing),
                    'extra': list(extra)
                })
            # Check column order (only if same count)
            elif columns != reference_columns:
                # Find position differences
                differences = []
                for idx, (ref_col, file_col) in enumerate(zip(reference_columns, columns)):
                    if ref_col != file_col:
                        differences.append(f"Position {idx}: '{ref_col}' in gw1, '{file_col}' in gw{i}")
                
                order_mismatches.append({
                    'file': f'gw{i}',
                    'differences': differences
                })
    
    # Generate diagnosis
    output_lines.append("\n=== Diagnosis Result ===\n")
    
    # Stage 1: Column Count Check
    output_lines.append("Stage 1: Column Count Check")
    if not count_mismatches:
        output_lines.append(f" All files have {reference_count} columns (same as gw1.csv)\n")
    else:
        output_lines.append(f" Column count differences found:")
        for mismatch in count_mismatches:
            output_lines.append(f"  - {mismatch['file']}: {mismatch['count']} columns (expected {reference_count})")
            if mismatch['missing']:
                output_lines.append(f"    Missing columns: {mismatch['missing']}")
            if mismatch['extra']:
                output_lines.append(f"    Extra columns: {mismatch['extra']}")
        files_with_correct_count = 38 - len(count_mismatches)
        output_lines.append(f"  - {files_with_correct_count}/38 files have correct count ({reference_count} columns)\n")
    
    # Stage 2: Column Order Check
    output_lines.append("Stage 2: Column Order Check")
    if count_mismatches:
        files_to_check = 38 - len(count_mismatches)
        output_lines.append(f"(Only checking {files_to_check} files with correct column count)")
    
    if not order_mismatches:
        if count_mismatches:
            files_matching = 38 - len(count_mismatches)
            output_lines.append(f" All {files_matching} files with correct count have matching column order\n")
        else:
            output_lines.append(" All files have matching column order\n")
    else:
        output_lines.append(" Found column order differences:")
        for mismatch in order_mismatches:
            output_lines.append(f"  - {mismatch['file']}:")
            for diff in mismatch['differences']:
                output_lines.append(f"    {diff}")
        output_lines.append("")
    
    # Overall summary
    total_matching = 38 - len(count_mismatches) - len(order_mismatches)
    output_lines.append(f"Overall: {total_matching}/38 files match gw1.csv perfectly")
    
    # Write to file
    output_file = 'gw_column_check_report.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(output_lines))
    
    print(f"Report saved to {output_file}")
    print("\n" + '\n'.join(output_lines[-15:]))  # Print summary to console

check_column_consistency()

Report saved to gw_column_check_report.txt

  - gw34: 48 columns (expected 41)
    Extra columns: ['mng_loss', 'mng_draw', 'mng_underdog_win', 'mng_goals_scored', 'mng_clean_sheets', 'mng_underdog_draw', 'mng_win']
  - gw35: 48 columns (expected 41)
    Extra columns: ['mng_loss', 'mng_draw', 'mng_underdog_win', 'mng_goals_scored', 'mng_clean_sheets', 'mng_underdog_draw', 'mng_win']
  - gw36: 48 columns (expected 41)
    Extra columns: ['mng_loss', 'mng_draw', 'mng_underdog_win', 'mng_goals_scored', 'mng_clean_sheets', 'mng_underdog_draw', 'mng_win']
  - gw37: 48 columns (expected 41)
    Extra columns: ['mng_loss', 'mng_draw', 'mng_underdog_win', 'mng_goals_scored', 'mng_clean_sheets', 'mng_underdog_draw', 'mng_win']
  - gw38: 48 columns (expected 41)
    Extra columns: ['mng_loss', 'mng_draw', 'mng_underdog_win', 'mng_goals_scored', 'mng_clean_sheets', 'mng_underdog_draw', 'mng_win']
  - 21/38 files have correct count (41 columns)

Stage 2: Column Order Check
(Only checking 21 files 

## Step 2: Merge GW Files

**Solution**: Merge gw1 to gw38 **without** the excessive columns found from gw22 to gw38.

**Process**:
1. Validate column consistency after removing extra columns
2. Merge all files with proper column alignment
3. Add 'gw' column to identify gameweek

**Extra columns to remove** (from gw22-gw38): `mng_clean_sheets`, `mng_draw`, `mng_goals_scored`, `mng_loss`, `mng_underdog_draw`, `mng_underdog_win`, `mng_win`

In [6]:
def validate_and_merge_gw_files():
    extra_columns = [
        'mng_clean_sheets', 'mng_draw', 'mng_goals_scored', 'mng_loss',
        'mng_underdog_draw', 'mng_underdog_win', 'mng_win'
    ]
    
    reference_file = 'gw_vaa/gw1.csv'
    with open(reference_file, 'r', encoding='utf-8') as f:
        reference_columns = next(csv.reader(f))
    
    print(f"Reference (gw1): {len(reference_columns)} columns")
    print(f"Expected columns: {reference_columns}\n")
    
    print("="*60)
    print("STEP 1: Validating column order for gw22-gw38")
    print("="*60)
    
    validation_passed = True
    column_indices_to_keep = {}
    
    for i in range(22, 39):
        filename = f'gw_vaa/gw{i}.csv'
        with open(filename, 'r', encoding='utf-8') as f:
            columns = next(csv.reader(f))
        
        filtered_columns = [col for col in columns if col not in extra_columns]
        indices_to_keep = [idx for idx, col in enumerate(columns) if col not in extra_columns]
        column_indices_to_keep[i] = indices_to_keep
        
        if filtered_columns == reference_columns:
            print(f" gw{i}: Column order matches after removing extra columns")
        else:
            print(f" gw{i}: Column order DOES NOT match!")
            validation_passed = False
    
    print("\n" + "="*60)
    if validation_passed:
        print(" VALIDATION PASSED: All files have matching column order")
    else:
        print(" VALIDATION FAILED: Column order mismatch detected")
        print("Merge operation ABORTED.")
        return False
    print("="*60)
    
    print("\n" + "="*60)
    print("STEP 2: Merging all files into gw_merged.csv")
    print("="*60)
    
    output_file = 'gw_vaa/gw_merged.csv'
    total_rows = 0
    
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        header_with_gw = reference_columns + ['gw']
        writer.writerow(header_with_gw)
        print(f" Header written ({len(header_with_gw)} columns, including 'gw')")
        
        for i in range(1, 39):
            filename = f'gw_vaa/gw{i}.csv'
            file_rows = 0
            
            with open(filename, 'r', encoding='utf-8') as infile:
                reader = csv.reader(infile)
                next(reader)
                
                if i <= 21:
                    for row in reader:
                        row_with_gw = row + [str(i)]
                        writer.writerow(row_with_gw)
                        file_rows += 1
                        total_rows += 1
                else:
                    indices = column_indices_to_keep[i]
                    for row in reader:
                        filtered_row = [row[idx] for idx in indices] + [str(i)]
                        writer.writerow(filtered_row)
                        file_rows += 1
                        total_rows += 1
            
            print(f" gw{i}: {file_rows} rows added")
    
    print("\n" + "="*60)
    print("MERGE COMPLETED SUCCESSFULLY")
    print("="*60)
    print(f"Output file: {output_file}")
    print(f"Total rows: {total_rows} (excluding header)")
    print(f"Total columns: {len(header_with_gw)} (including 'gw' column)")
    
    return True

validate_and_merge_gw_files()

Reference (gw1): 41 columns
Expected columns: ['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified', 'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score', 'threat', 'total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards']

STEP 1: Validating column order for gw22-gw38
 gw22: Column order matches after removing extra columns
 gw23: Column order matches after removing extra columns
 gw24: Column order matches after removing extra columns
 gw25: Column order matches after removing extra columns
 gw26: Column order matches after removing extra columns
 gw27: Column order matches after removing 

True

## Step 3: Filter GW Merged Columns

**Problem**: After merging, there are too many variables unnecessary for the study.

**Solution**: Keep only the following variables: `name`, `position`, `team`, `xP`, `minutes`, `starts`, `total_points`, `value`, `gw`

**Row Count After This Step**:
- gw_merged_filtered.csv: **27,605 rows**
- playerstats_mapped_filtered.csv: **27,658 rows**

A serious cross-referencing has to be done.

In [7]:
# Define the columns to extract
columns_to_extract = [
    'name',
    'position',
    'team',
    'xP',
    'minutes',
    'starts',
    'total_points',
    'value',
    'gw'
]

# Read only the specified columns from the CSV file
df = pd.read_csv('gw_vaa/gw_merged.csv', usecols=columns_to_extract)

# Save the filtered data to a new CSV file
df.to_csv('gw_vaa/gw_merged_filtered.csv', index=False)

print(f"Successfully extracted {len(columns_to_extract)} columns from gw_merged.csv")
print(f"Output saved to: gw_vaa/gw_merged_filtered.csv")
print(f"Number of rows: {len(df)}")

Successfully extracted 9 columns from gw_merged.csv
Output saved to: gw_vaa/gw_merged_filtered.csv
Number of rows: 27605


## Step 4: Check for Duplicate (gw, name) Pairs in VAA Dataset

**Investigation**: Check if there exist repeated pairs of (gw=i, name=j) in the VAA dataset.

**Finding**: Duplicates exist, and most of them occurred in GW 24, 25, 32, and 33.

In [8]:
# Check VAA file for duplicates
vaa_file_path = 'gw_vaa/gw_merged_filtered.csv'
print(f"\n{'='*80}")
print(f"Checking: VAA gw_merged_filtered.csv")
print(f"File: {vaa_file_path}")
print(f"{'='*80}")

df_vaa = pd.read_csv(vaa_file_path)
total_rows = len(df_vaa)
print(f"\nTotal rows: {total_rows:,}")

# Group by (gw, name) and count occurrences
duplicates = df_vaa.groupby(['gw', 'name']).size().reset_index(name='count')
duplicates = duplicates[duplicates['count'] > 1].sort_values('count', ascending=False)

if len(duplicates) == 0:
    print(f"\n PASS: No duplicate (gw, name) combinations found!")
else:
    print(f"\n Found {len(duplicates)} duplicate (gw, name) combinations!")
    
    # Distribution by gameweek
    dup_by_gw = duplicates.groupby(df_vaa.loc[df_vaa.set_index(['gw', 'name']).index.isin(
        pd.MultiIndex.from_frame(duplicates[['gw', 'name']])
    ), 'gw']).size().reset_index(name='dup_count')
    
    print(f"\nDuplicate distribution by GW:")
    for _, row in duplicates.groupby(duplicates['gw'].map(lambda x: x)).size().reset_index(name='pairs').iterrows():
        print(f"  GW {row[0]}: {row['pairs']} duplicate pairs")
    
    print(f"\nSample of duplicate pairs:")
    print(duplicates.head(10))


Checking: VAA gw_merged_filtered.csv
File: gw_vaa/gw_merged_filtered.csv

Total rows: 27,605

 Found 374 duplicate (gw, name) combinations!

Duplicate distribution by GW:
  GW 24: 69 duplicate pairs
  GW 25: 79 duplicate pairs
  GW 32: 71 duplicate pairs
  GW 33: 155 duplicate pairs

Sample of duplicate pairs:
       gw                       name  count
23373  33                 Zach Marsh      2
15667  24         Abdoulaye Doucouré      2
15687  24        Alexis Mac Allister      2
15694  24      Alisson Ramses Becker      2
15699  24                Amara Nallo      2
15705  24           Andrew Robertson      2
23270  33               Scott Carson      2
23263  33        Samuel Iling-Junior      2
23256  33  Salah-Eddine Oulad M'hand      2
23255  33      Rúben Gato Alves Dias      2


C:\Users\user\AppData\Local\Temp\ipykernel_60580\72219451.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"  GW {row[0]}: {row['pairs']} duplicate pairs")


## Step 5: Analyze Duplicate Rows

**Investigation**: Further analysis shows that:
1. The count of occurrence of all repeated pairs is **2** (each duplicated pair has exactly 2 rows)
2. Those duplicated pairs have **different data**

**Explanation**: By inspection of the unfiltered merged dataset, in GW 24, 25, 32, 33, some clubs had **two fixtures (matches)** rather than one because of rescheduling due to weather or other reasons. This is confirmed by different kickoff times and enemy team codes.

In [9]:
def analyze_duplicate_rows(file_path, gw_col, name_col):
    print(f"\n{'='*80}")
    print(f"DUPLICATE ROW ANALYSIS")
    print(f"File: {file_path}")
    print(f"Checking if duplicate (gw, name) pairs have identical or different rows")
    print(f"{'='*80}")
    
    df = pd.read_csv(file_path)
    
    print(f"\nTotal rows in file: {len(df):,}")
    print(f"Total columns: {len(df.columns)}")
    
    # Find duplicate (gw, name) pairs
    duplicate_pairs = df.groupby([gw_col, name_col]).size().reset_index(name='count')
    duplicate_pairs = duplicate_pairs[duplicate_pairs['count'] > 1]
    
    if len(duplicate_pairs) == 0:
        print("\n No duplicate (gw, name) pairs found!")
        return
    
    print(f"\nFound {len(duplicate_pairs)} duplicate (gw, name) pairs")
    print(f"Total rows involved: {duplicate_pairs['count'].sum()}")
    
    # Analyze each duplicate pair
    identical_duplicates = []
    different_duplicates = []
    
    print(f"\nAnalyzing duplicate pairs...")
    
    for idx, row in duplicate_pairs.iterrows():
        gw_val = row[gw_col]
        name_val = row[name_col]
        count = row['count']
        
        # Get all rows for this (gw, name) pair
        mask = (df[gw_col] == gw_val) & (df[name_col] == name_val)
        duplicate_rows = df[mask]
        
        # Check if all rows are identical
        first_row = duplicate_rows.iloc[0]
        all_identical = True
        
        for i in range(1, len(duplicate_rows)):
            if not duplicate_rows.iloc[i].equals(first_row):
                all_identical = False
                break
        
        if all_identical:
            identical_duplicates.append({
                'gw': gw_val,
                'name': name_val,
                'count': count
            })
        else:
            different_duplicates.append({
                'gw': gw_val,
                'name': name_val,
                'count': count,
                'rows': duplicate_rows
            })
    
    # Report results
    print(f"\n{'='*80}")
    print("ANALYSIS RESULTS")
    print(f"{'='*80}")
    
    print(f"\n Identical duplicates: {len(identical_duplicates)} pairs")
    print(f"  (These are exact copies - can be safely deduplicated)")
    
    print(f"\n Different data duplicates: {len(different_duplicates)} pairs")
    if len(different_duplicates) > 0:
        print(f"  (These have different values - need aggregation)")
    
    # Show detailed analysis of different duplicates
    if len(different_duplicates) > 0:
        print(f"\n{'='*80}")
        print("DIFFERENT DATA DUPLICATES - DETAILED ANALYSIS")
        print(f"{'='*80}")
        
        for i, dup in enumerate(different_duplicates[:5]):  # Show first 5
            print(f"\n--- Duplicate #{i+1}: GW {dup['gw']}, {dup['name']} ---")
            print(f"Number of occurrences: {dup['count']}")
            
            # Show all rows for this duplicate
            rows = dup['rows']
            print(f"\nAll {len(rows)} rows for this (gw, name) pair:")
            print(rows.to_string(index=False))
            
            # Find which columns differ
            print(f"\nColumns with different values:")
            differing_cols = []
            for col in rows.columns:
                if rows[col].nunique() > 1:
                    differing_cols.append(col)
                    unique_vals = rows[col].unique()
                    print(f"  - {col}: {unique_vals}")
            
            print("-" * 80)
        
        if len(different_duplicates) > 5:
            print(f"\n... and {len(different_duplicates) - 5} more pairs with different data")
    
    print(f"\n{'='*80}\n")

analyze_duplicate_rows(
    file_path='gw_vaa/gw_merged_filtered.csv',
    gw_col='gw',
    name_col='name'
)


DUPLICATE ROW ANALYSIS
File: gw_vaa/gw_merged_filtered.csv
Checking if duplicate (gw, name) pairs have identical or different rows

Total rows in file: 27,605
Total columns: 9

Found 374 duplicate (gw, name) pairs
Total rows involved: 748

Analyzing duplicate pairs...

ANALYSIS RESULTS

 Identical duplicates: 202 pairs
  (These are exact copies - can be safely deduplicated)

 Different data duplicates: 172 pairs
  (These have different values - need aggregation)

DIFFERENT DATA DUPLICATES - DETAILED ANALYSIS

--- Duplicate #1: GW 24, Abdoulaye Doucouré ---
Number of occurrences: 2

All 2 rows for this (gw, name) pair:
              name position    team  xP  minutes  starts  total_points  value  gw
Abdoulaye Doucouré      MID Everton 4.2       90       1             8     51  24
Abdoulaye Doucouré      MID Everton 4.2       90       1            -1     51  24

Columns with different values:
  - total_points: [ 8 -1]
---------------------------------------------------------------------

---

# Part 3: Problem from Cross-Referencing

Before combining the two data sources, we need to handle the duplicates and verify data consistency.

## Step 1: Verify Duplicate Points

**Hypothesis**: In VAA's dataset, for those players who have duplicated pair data rows (gw=i, name=j), the **sum of 'total_points' from two matches in that gw** should equal the **'event_points' at (gw=i, name=j) in OLB's dataset**.

This will verify data consistency between the two sources.

In [10]:
# Load the datasets
print("Loading datasets...")
vaa_df = pd.read_csv('gw_vaa/gw_merged_filtered.csv')
olb_df = pd.read_csv('gw_olb/playerstats_mapped_filtered.csv')

print(f"VAA dataset: {len(vaa_df)} rows")
print(f"OLB dataset: {len(olb_df)} rows")
print()

# Find duplicated (gw, name) pairs in VAA dataset
print("Identifying duplicated (gw, name) pairs in VAA dataset...")
duplicated_mask = vaa_df.duplicated(subset=['gw', 'name'], keep=False)
duplicated_df = vaa_df[duplicated_mask].copy()

print(f"Found {len(duplicated_df)} rows with duplicated (gw, name) pairs")
print()

# Count unique duplicated pairs
unique_pairs = duplicated_df.groupby(['gw', 'name']).size()
print(f"Number of unique duplicated (gw, name) pairs: {len(unique_pairs)}")
print()

# Check that all duplicated pairs appear exactly twice
pair_counts = unique_pairs.value_counts()
print("Distribution of duplicate counts:")
print(pair_counts)
print()

if len(pair_counts) > 1 or (len(pair_counts) == 1 and pair_counts.index[0] != 2):
    print("WARNING: Not all duplicated pairs appear exactly twice!")
    print("Pairs with counts != 2:")
    non_two_pairs = unique_pairs[unique_pairs != 2]
    print(non_two_pairs)
    print()

# Sum total_points for each duplicated (gw, name) pair
print("Summing total_points for each duplicated pair...")
vaa_summed = duplicated_df.groupby(['gw', 'name'])['total_points'].sum().reset_index()
vaa_summed.columns = ['gw', 'name', 'vaa_sum_total_points']

print(f"Created {len(vaa_summed)} summed records")
print()

# Merge with OLB dataset
print("Merging with OLB dataset...")
merged = vaa_summed.merge(
    olb_df[['gw', 'player_name', 'event_points']],
    left_on=['gw', 'name'],
    right_on=['gw', 'player_name'],
    how='left'
)

print(f"Merged dataset has {len(merged)} rows")
print()

# Check for unmatched records
unmatched = merged[merged['event_points'].isna()]
if len(unmatched) > 0:
    print(f"WARNING: {len(unmatched)} duplicated pairs from VAA not found in OLB dataset:")
    print(unmatched[['gw', 'name', 'vaa_sum_total_points']])
    print()

# Remove unmatched records for comparison
merged_matched = merged[merged['event_points'].notna()].copy()

# Compare values (exact integer comparison)
print("Comparing summed total_points with event_points...")
merged_matched['match'] = merged_matched['vaa_sum_total_points'] == merged_matched['event_points']

# Count matches and mismatches
num_matches = merged_matched['match'].sum()
num_mismatches = (~merged_matched['match']).sum()

print("=" * 70)
print("VERIFICATION RESULTS")
print("=" * 70)
print(f"Total duplicated (gw, name) pairs analyzed: {len(merged_matched)}")
print(f"Exact matches (VAA sum == OLB event_points): {num_matches}")
print(f"Mismatches (VAA sum != OLB event_points): {num_mismatches}")
print()

if num_mismatches > 0:
    print("HYPOTHESIS VERIFICATION: FAILED")
    print(f"Found {num_mismatches} mismatches")
    print()
    print("Details of mismatches:")
    print("-" * 70)
    mismatches = merged_matched[~merged_matched['match']][['gw', 'name', 'vaa_sum_total_points', 'event_points']]
    mismatches['difference'] = mismatches['vaa_sum_total_points'] - mismatches['event_points']
    print(mismatches.to_string(index=False))
else:
    print("HYPOTHESIS VERIFICATION: PASSED")
    print("All duplicated pairs' summed total_points match event_points exactly!")

print("=" * 70)

Loading datasets...
VAA dataset: 27605 rows
OLB dataset: 27657 rows

Identifying duplicated (gw, name) pairs in VAA dataset...
Found 748 rows with duplicated (gw, name) pairs

Number of unique duplicated (gw, name) pairs: 374

Distribution of duplicate counts:
2    374
Name: count, dtype: int64

Summing total_points for each duplicated pair...
Created 374 summed records

Merging with OLB dataset...
Merged dataset has 374 rows

Comparing summed total_points with event_points...
VERIFICATION RESULTS
Total duplicated (gw, name) pairs analyzed: 374
Exact matches (VAA sum == OLB event_points): 374
Mismatches (VAA sum != OLB event_points): 0

HYPOTHESIS VERIFICATION: PASSED
All duplicated pairs' summed total_points match event_points exactly!


## Step 2: Merge Duplicate GWs

**Solution**: For GW 24, 25, 32, and 33, for those players having two matches, the following data manipulation will be done:

**Aggregation Rules** (assuming name, position, team are unchanged):
- **Sum**: xP, total_points
- **Max**: starts
- **Average & round**: value (to the closest 0.1 Million), minutes (to nearest integer)

Output as `gw_merged_filtered_nodup.csv`

**After this step**:
- VAA dataset will have **27,231 rows**
- OLB dataset still has **27,658 rows**

In [11]:
# Load the dataset
print("Loading gw_merged_filtered.csv...")
df = pd.read_csv('gw_vaa/gw_merged_filtered.csv')

print(f"Total rows: {len(df)}")
print(f"Columns: {', '.join(df.columns)}")
print()

# Define target gameweeks
target_gws = [24, 25, 32, 33]

# Identify duplicates in target gameweeks
print(f"Identifying duplicates in GWs {target_gws}...")
target_gw_df = df[df['gw'].isin(target_gws)].copy()
duplicated_mask = target_gw_df.duplicated(subset=['gw', 'name'], keep=False)
duplicates_df = target_gw_df[duplicated_mask].copy()

print(f"Found {len(duplicates_df)} duplicate rows in target GWs")
print(f"Number of unique duplicate pairs: {len(duplicates_df.groupby(['gw', 'name']))}")
print()

# Aggregate duplicates
if len(duplicates_df) > 0:
    print("Aggregating duplicate rows...")
    
    # Define aggregation rules
    agg_dict = {
        'name': 'first',
        'position': 'first',
        'team': 'first',
        'xP': 'sum',
        'minutes': lambda x: round(x.mean()),  # Average and round to nearest integer
        'starts': 'max',
        'total_points': 'sum',
        'value': lambda x: round(x.mean()),  # Average and round to nearest integer
        'gw': 'first'
    }
    
    aggregated_df = duplicates_df.groupby(['gw', 'name'], as_index=False).agg(agg_dict)
    
    print(f"Aggregated to {len(aggregated_df)} rows")
    print()
else:
    aggregated_df = pd.DataFrame(columns=df.columns)
    print("No duplicates found to aggregate")
    print()

# Get non-duplicate rows
print("Collecting non-duplicate rows...")

# Non-duplicates from target GWs
non_dup_target_gws = target_gw_df[~duplicated_mask].copy()

# All rows from other GWs
other_gws_df = df[~df['gw'].isin(target_gws)].copy()

print(f"Non-duplicate rows from target GWs: {len(non_dup_target_gws)}")
print(f"Rows from other GWs: {len(other_gws_df)}")
print()

# Combine all parts
print("Combining datasets...")
result_df = pd.concat([aggregated_df, non_dup_target_gws, other_gws_df], ignore_index=True)

# Sort by gw and name for consistency
result_df = result_df.sort_values(['gw', 'name']).reset_index(drop=True)

print(f"Final dataset rows: {len(result_df)}")
print()

# Verify column order
expected_columns = ['name', 'position', 'team', 'xP', 'minutes', 'starts', 'total_points', 'value', 'gw']
result_df = result_df[expected_columns]

# Save the result
output_path = 'gw_vaa/gw_merged_filtered_nodup.csv'
print(f"Saving to {output_path}...")
result_df.to_csv(output_path, index=False)

print("Done!")
print()

# Summary statistics
print("=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Input rows:  {len(df):,}")
print(f"Output rows: {len(result_df):,}")
print(f"Rows removed (duplicates merged): {len(df) - len(result_df):,}")
print()

# Show sample of aggregated rows if any
if len(aggregated_df) > 0:
    print("Sample of aggregated duplicate rows:")
    print(aggregated_df.head(10).to_string(index=False))
    print()

print("=" * 70)

Loading gw_merged_filtered.csv...
Total rows: 27605
Columns: name, position, team, xP, minutes, starts, total_points, value, gw

Identifying duplicates in GWs [24, 25, 32, 33]...
Found 748 duplicate rows in target GWs
Number of unique duplicate pairs: 374

Aggregating duplicate rows...
Aggregated to 374 rows

Non-duplicate rows from target GWs: 2764
Rows from other GWs: 24093

Combining datasets...
Final dataset rows: 27231

Saving to gw_vaa/gw_merged_filtered_nodup.csv...
Done!

SUMMARY
Input rows:  27,605
Output rows: 27,231
Rows removed (duplicates merged): 374

Sample of aggregated duplicate rows:
                 name position      team   xP  minutes  starts  total_points  value  gw
   Abdoulaye Doucouré      MID   Everton  8.4       90       1             7     51  24
  Alexis Mac Allister      MID Liverpool 17.4       75       1            10     62  24
Alisson Ramses Becker       GK Liverpool 21.4       90       1            10     55  24
          Amara Nallo      DEF Liverpoo

## Step 3: Verify No Duplicates After Merge

**Verification**: Before merging datasets, verify that all (gw, name) pairs are now unique in both datasets.

This is important because (gw, name) will be used as the unique identifier for merging.

In [12]:
def check_duplicates_both(file1_path, file2_path):
    """Check both files for duplicate (gw, name) pairs."""
    
    print("\n" + "="*80)
    print("DUPLICATE (GW, NAME) VALIDATION CHECK")
    print("="*80)
    
    # Check VAA file
    print(f"\n{'='*80}")
    print(f"Checking: gw_merged_filtered_nodup.csv")
    print(f"File: {file1_path}")
    print(f"{'='*80}")
    
    df1 = pd.read_csv(file1_path)
    total_rows1 = len(df1)
    print(f"\nTotal rows: {total_rows1:,}")
    
    duplicates1 = df1.groupby(['gw', 'name']).size().reset_index(name='count')
    duplicates1 = duplicates1[duplicates1['count'] > 1].sort_values('count', ascending=False)
    
    if len(duplicates1) == 0:
        print(f"\n✓ PASS: No duplicate (gw, name) combinations found!")
        unique_combinations1 = len(df1.groupby(['gw', 'name']))
        print(f"  Total unique (gw, name) combinations: {unique_combinations1:,}")
        file1_ok = True
    else:
        print(f"\n✗ FAIL: Found {len(duplicates1)} duplicate (gw, name) combinations!")
        file1_ok = False
    
    # Check OLB file
    print(f"\n{'='*80}")
    print(f"Checking: playerstats_mapped_filtered.csv")
    print(f"File: {file2_path}")
    print(f"{'='*80}")
    
    df2 = pd.read_csv(file2_path)
    total_rows2 = len(df2)
    print(f"\nTotal rows: {total_rows2:,}")
    
    duplicates2 = df2.groupby(['gw', 'player_name']).size().reset_index(name='count')
    duplicates2 = duplicates2[duplicates2['count'] > 1].sort_values('count', ascending=False)
    
    if len(duplicates2) == 0:
        print(f"\n✓ PASS: No duplicate (gw, name) combinations found!")
        unique_combinations2 = len(df2.groupby(['gw', 'player_name']))
        print(f"  Total unique (gw, name) combinations: {unique_combinations2:,}")
        file2_ok = True
    else:
        print(f"\n✗ FAIL: Found {len(duplicates2)} duplicate (gw, name) combinations!")
        file2_ok = False
    
    # Final summary
    print(f"\n{'='*80}")
    print("FINAL SUMMARY")
    print(f"{'='*80}")
    
    print(f"\ngw_merged_filtered_nodup.csv:    {'PASS ✓' if file1_ok else 'FAIL ✗'}")
    print(f"playerstats_mapped_filtered.csv: {'PASS ✓' if file2_ok else 'FAIL ✗'}")
    
    if file1_ok and file2_ok:
        print(f"\n{'='*80}")
        print("✓ ALL CHECKS PASSED - Both files have no duplicate (gw, name) combinations")
        print(f"{'='*80}\n")
    else:
        print(f"\n{'='*80}")
        print("✗ VALIDATION FAILED - Please review the duplicates listed above")
        print(f"{'='*80}\n")

check_duplicates_both(
    'gw_vaa/gw_merged_filtered_nodup.csv',
    'gw_olb/playerstats_mapped_filtered.csv'
)


DUPLICATE (GW, NAME) VALIDATION CHECK

Checking: gw_merged_filtered_nodup.csv
File: gw_vaa/gw_merged_filtered_nodup.csv

Total rows: 27,231

✓ PASS: No duplicate (gw, name) combinations found!
  Total unique (gw, name) combinations: 27,231

Checking: playerstats_mapped_filtered.csv
File: gw_olb/playerstats_mapped_filtered.csv

Total rows: 27,657

✓ PASS: No duplicate (gw, name) combinations found!
  Total unique (gw, name) combinations: 27,657

FINAL SUMMARY

gw_merged_filtered_nodup.csv:    PASS ✓
playerstats_mapped_filtered.csv: PASS ✓

✓ ALL CHECKS PASSED - Both files have no duplicate (gw, name) combinations



## Step 4: Match Datasets by (name, gw) Pairs

**Problem**: There are 27,658 - 27,231 = **427 row difference** between the two datasets.

**Investigation**: Match the two datasets by (name, gw) pair and identify which pairs exist in only one dataset.

**Expected Results** (from problem sheet):
- Total unique (name, gw) pairs in OLB dataset: 27,657
- Total unique (name, gw) pairs in VAA dataset: 27,231
- Matching pairs (in both datasets): 27,222
- Match percentage: **98.43%**
- Pairs only in OLB dataset: 435
- Pairs only in VAA dataset: 9

**Decision**: Since both datasets complement each other and there's a 98.43% match percentage (quite good for real-life datasets), we'll keep only the matching pairs for integration.

In [13]:
def match_datasets():
    """
    Match two datasets by (name, gw) pairs and identify non-matching pairs.
    """
    
    # Load only the necessary columns from each dataset
    print("Loading datasets...")
    olb_data = pd.read_csv('gw_olb/playerstats_mapped_filtered.csv', usecols=['player_name', 'gw'])
    vaa_data = pd.read_csv('gw_vaa/gw_merged_filtered_nodup.csv', usecols=['name', 'gw'])
    
    print(f"OLB data loaded: {len(olb_data)} rows")
    print(f"VAA data loaded: {len(vaa_data)} rows")
    
    # Create sets of (name, gw) tuples
    olb_pairs = set(zip(olb_data['player_name'], olb_data['gw']))
    vaa_pairs = set(zip(vaa_data['name'], vaa_data['gw']))
    
    # Find non-matching pairs
    only_in_olb = olb_pairs - vaa_pairs
    only_in_vaa = vaa_pairs - olb_pairs
    matching_pairs = olb_pairs & vaa_pairs
    
    # Calculate statistics
    total_olb = len(olb_pairs)
    total_vaa = len(vaa_pairs)
    total_matching = len(matching_pairs)
    match_percentage = (total_matching / max(total_olb, total_vaa)) * 100
    
    # Prepare report
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("DATASET MATCHING REPORT")
    report_lines.append("=" * 80)
    report_lines.append("")
    report_lines.append("SUMMARY STATISTICS")
    report_lines.append("-" * 80)
    report_lines.append(f"Total unique (name, gw) pairs in OLB dataset: {total_olb}")
    report_lines.append(f"Total unique (name, gw) pairs in VAA dataset: {total_vaa}")
    report_lines.append(f"Matching pairs (in both datasets):           {total_matching}")
    report_lines.append(f"Match percentage:                             {match_percentage:.2f}%")
    report_lines.append("")
    report_lines.append(f"Pairs only in OLB dataset:                    {len(only_in_olb)}")
    report_lines.append(f"Pairs only in VAA dataset:                    {len(only_in_vaa)}")
    report_lines.append("")
    
    # Add gameweek-level analysis
    report_lines.append("=" * 80)
    report_lines.append("NON-MATCHING PAIRS BY GAMEWEEK")
    report_lines.append("=" * 80)
    report_lines.append("")
    
    if only_in_olb:
        # Count pairs by gameweek for OLB
        gw_counts_olb = {}
        for name, gw in only_in_olb:
            gw_counts_olb[gw] = gw_counts_olb.get(gw, 0) + 1
        
        report_lines.append("PAIRS ONLY IN OLB (by GW):")
        report_lines.append("-" * 80)
        # Sort by count descending
        for gw, count in sorted(gw_counts_olb.items(), key=lambda x: (-x[1], x[0])):
            report_lines.append(f"  GW {gw:2d}: {count:3d} pairs")
        report_lines.append("")
    
    if only_in_vaa:
        # Count pairs by gameweek for VAA
        gw_counts_vaa = {}
        for name, gw in only_in_vaa:
            gw_counts_vaa[gw] = gw_counts_vaa.get(gw, 0) + 1
        
        report_lines.append("PAIRS ONLY IN VAA (by GW):")
        report_lines.append("-" * 80)
        # Sort by count descending
        for gw, count in sorted(gw_counts_vaa.items(), key=lambda x: (-x[1], x[0])):
            report_lines.append(f"  GW {gw:2d}: {count:3d} pairs")
        report_lines.append("")
    
    report_lines.append("=" * 80)
    report_lines.append("END OF REPORT")
    report_lines.append("=" * 80)
    
    # Print to console
    report_text = "\n".join(report_lines)
    print(report_text)
    
    # Save to file
    output_file = 'dataset_matching_report.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"\nReport saved to: {output_file}")
    
    return {
        'total_olb': total_olb,
        'total_vaa': total_vaa,
        'matching': total_matching,
        'only_in_olb': len(only_in_olb),
        'only_in_vaa': len(only_in_vaa),
        'match_percentage': match_percentage
    }

results = match_datasets()

Loading datasets...
OLB data loaded: 27657 rows
VAA data loaded: 27231 rows
DATASET MATCHING REPORT

SUMMARY STATISTICS
--------------------------------------------------------------------------------
Total unique (name, gw) pairs in OLB dataset: 27657
Total unique (name, gw) pairs in VAA dataset: 27231
Matching pairs (in both datasets):           27222
Match percentage:                             98.43%

Pairs only in OLB dataset:                    435
Pairs only in VAA dataset:                    9

NON-MATCHING PAIRS BY GAMEWEEK

PAIRS ONLY IN OLB (by GW):
--------------------------------------------------------------------------------
  GW 34: 155 pairs
  GW 29: 151 pairs
  GW 15:  63 pairs
  GW 22:  26 pairs
  GW 24:  12 pairs
  GW 20:  10 pairs
  GW  3:   7 pairs
  GW 21:   2 pairs
  GW 23:   2 pairs
  GW 27:   2 pairs
  GW 25:   1 pairs
  GW 26:   1 pairs
  GW 28:   1 pairs
  GW 30:   1 pairs
  GW 31:   1 pairs

PAIRS ONLY IN VAA (by GW):
--------------------------------------

## Step 5: Integrate Datasets

**Final Step**: Filter both datasets to keep only rows where (name, gw) exists in matching pairs, then merge them.

**Process**:
1. OLB: Keep only rows where (player_name, gw) is in matching pairs
2. VAA: Keep only rows where (name, gw) is in matching pairs
3. Merge on (name, gw)
4. Save as `integrated_df.csv`

In [14]:
def integrate_datasets():
    """
    Filter and integrate OLB and VAA datasets based on matching (name, gw) pairs.
    """
    
    print("Loading datasets...")
    # Load both datasets
    olb_data = pd.read_csv('gw_olb/playerstats_mapped_filtered.csv')
    vaa_data = pd.read_csv('gw_vaa/gw_merged_filtered_nodup.csv')
    
    print(f"OLB data loaded: {len(olb_data)} rows, {len(olb_data.columns)} columns")
    print(f"VAA data loaded: {len(vaa_data)} rows, {len(vaa_data.columns)} columns")
    
    # Create sets of (name, gw) tuples
    print("\nIdentifying matching pairs...")
    olb_pairs = set(zip(olb_data['player_name'], olb_data['gw']))
    vaa_pairs = set(zip(vaa_data['name'], vaa_data['gw']))
    
    # Find matching pairs (intersection)
    matching_pairs = olb_pairs & vaa_pairs
    
    print(f"Total matching pairs: {len(matching_pairs)}")
    print(f"Pairs to remove from OLB: {len(olb_pairs) - len(matching_pairs)}")
    print(f"Pairs to remove from VAA: {len(vaa_pairs) - len(matching_pairs)}")
    
    # Filter OLB dataset to keep only matching pairs
    print("\nFiltering OLB dataset...")
    olb_filtered = olb_data[
        olb_data.apply(lambda row: (row['player_name'], row['gw']) in matching_pairs, axis=1)
    ].copy()
    
    # Rename player_name to name in OLB dataset
    olb_filtered.rename(columns={'player_name': 'name'}, inplace=True)
    
    print(f"OLB filtered: {len(olb_filtered)} rows remaining")
    
    # Filter VAA dataset to keep only matching pairs
    print("Filtering VAA dataset...")
    vaa_filtered = vaa_data[
        vaa_data.apply(lambda row: (row['name'], row['gw']) in matching_pairs, axis=1)
    ].copy()
    
    print(f"VAA filtered: {len(vaa_filtered)} rows remaining")
    
    # Merge the two datasets on (name, gw)
    print("\nMerging datasets...")
    integrated_df = pd.merge(
        olb_filtered,
        vaa_filtered,
        on=['name', 'gw'],
        how='inner',
        suffixes=('_olb', '_vaa')
    )
    
    print(f"Integrated dataset: {len(integrated_df)} rows, {len(integrated_df.columns)} columns")
    
    # Display column information
    print("\nColumn names in integrated dataset:")
    print(integrated_df.columns.tolist())
    
    # Save the integrated dataset
    output_file = 'integrated_df.csv'
    integrated_df.to_csv(output_file, index=False)
    print(f"\n✓ Integrated dataset saved to: {output_file}")
    
    # Display summary statistics
    print("\n" + "=" * 80)
    print("INTEGRATION SUMMARY")
    print("=" * 80)
    print(f"Original OLB rows:        {len(olb_data)}")
    print(f"Original VAA rows:        {len(vaa_data)}")
    print(f"Rows removed from OLB:    {len(olb_data) - len(olb_filtered)}")
    print(f"Rows removed from VAA:    {len(vaa_data) - len(vaa_filtered)}")
    print(f"Final integrated rows:    {len(integrated_df)}")
    print(f"Total columns:            {len(integrated_df.columns)}")
    print("=" * 80)
    
    return integrated_df

integrated_df = integrate_datasets()

Loading datasets...
OLB data loaded: 27657 rows, 9 columns
VAA data loaded: 27231 rows, 9 columns

Identifying matching pairs...
Total matching pairs: 27222
Pairs to remove from OLB: 435
Pairs to remove from VAA: 9

Filtering OLB dataset...
OLB filtered: 27222 rows remaining
Filtering VAA dataset...
VAA filtered: 27222 rows remaining

Merging datasets...
Integrated dataset: 27222 rows, 16 columns

Column names in integrated dataset:
['name', 'status', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'now_cost', 'event_points', 'ep_next', 'ep_this', 'gw', 'position', 'team', 'xP', 'minutes', 'starts', 'total_points', 'value']

✓ Integrated dataset saved to: integrated_df.csv

INTEGRATION SUMMARY
Original OLB rows:        27657
Original VAA rows:        27231
Rows removed from OLB:    435
Rows removed from VAA:    9
Final integrated rows:    27222
Total columns:            16


---

# Data Integration Complete!

The integrated dataset has been successfully created as `integrated_df.csv`.

This dataset combines:
- **OLB data**: status, chance_of_playing_next_round, chance_of_playing_this_round, now_cost, event_points, ep_next, ep_this
- **VAA data**: position, team, xP, minutes, starts, total_points, value

All rows represent unique (name, gw) pairs that exist in both original datasets, ensuring data quality and consistency.

**Next Steps**: The integrated data can now be cleaned and processed in the `data_cleaning.ipynb` notebook.